In this demo, we demonstrate how to use the MaxEnt_Reg script. We will be going over how to do the following:

1. Reading in the MaxEnt data and constraint information
2. Running the model using different gradient descent techniques (without regularization)
3. How to add regularization 
    * Target prior ```TGTPrior``` 
    * Difference prior ```DIFPrior```

In [37]:
## Good old NumPy
import numpy as np

## Specialized classes
from utils.MaxEnt import MaxEnt, compute_probabilities
from utils.Regularization import TGTPrior, DIFPrior

## Utility function for reading in constraint and data information
from utils.OTSoft_file_reader import get_info


In [59]:
import csv

def prob_prediction_to_csv(file_name, underlying_forms, candidates, violations, weights):
    with open(file_name, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["UR/Candidate", "Probability"])
        for t, vio in enumerate(violations):
            P = compute_probabilities(weights.squeeze(), vio)
            writer.writerow([underlying_forms[t]])
            
            for c, _ in enumerate(candidates[t]):
                writer.writerow([candidates[t][c], f"{round(P[c]*100, 1)}%"])


For this demonstration, we will be looking at the Hayes Pseudo-Korean data

In [28]:
## Read in data and constraint information
constraint_names, underlying_forms, candidates, violations, observed_probs = get_info(
    "toy_datasets/HayesPseudoKorean-RichBase.txt"
)

## Initialize a MaxEnt object
me = MaxEnt(constraint_names)

## Print basic information
print(f"Constraint names and their initial weights:")
for cn in zip(constraint_names, me.cws):
    print(f'{cn[0]:<20} {round(cn[1][0], 2)}')
print()


Constraint names and their initial weights:
Ident (asp)          7.5
Ident (voice)        2.25
Ident (asp)/_V       3.97
Ident (voice)/_V     3.16
*[+v][-v][+v]        8.08
*dh                  8.89
*[-son/+voice]       4.94
*aspiration          7.28



In [60]:
## Perform learning without regularization and returns the final weights
new_weights, learning_history = me.SGD_learn(violations=violations, observed_prob=observed_probs)

print("Weights after learning:")
for cn in zip(constraint_names, new_weights):
    print(f'{cn[0]:<20} {round(cn[1][0], 2)}')
print()

file_name = "results/no_reg_pred.csv"
prob_prediction_to_csv(file_name, underlying_forms, candidates, violations, new_weights)
print(f"Predicted probabilities saved to {file_name}")

Weights after learning:
Ident (asp)          8.01
Ident (voice)        2.3
Ident (asp)/_V       4.48
Ident (voice)/_V     3.19
*[+v][-v][+v]        8.07
*dh                  8.89
*[-son/+voice]       4.98
*aspiration          6.77

Predicted probabilities saved to results/no_reg_pred.csv


In [62]:
## M >> F TGT
cns = constraint_names
grs = [
    ["Ident (asp)", "Ident (voice)", "Ident (asp)/_V", "Ident (voice)/_V"], 
    ["*[+v][-v][+v]", "*dh", "*[-son/+voice]","*aspiration"]
]
mus = [1, 10]
sms = [1, 1]
target_prior = TGTPrior(cns, grs, mus, sms)

## Perform learning with regularization and returns the final weights
new_weights, learning_history = me.SGD_learn(violations, observed_probs, 1, 10000, 0.05, target_prior)

print("Weights after learning:")
for cn in zip(cns, new_weights):
    print(f'{cn[0]:<20} {round(cn[1][0], 2)}')
print()

file_name = "results/TGT_pred.csv"
prob_prediction_to_csv(file_name, underlying_forms, candidates, violations, new_weights)
print(f"Predicted probabilities saved to {file_name}")

Weights after learning:
Ident (asp)          1.68
Ident (voice)        1.05
Ident (asp)/_V       1.68
Ident (voice)/_V     1.05
*[+v][-v][+v]        10.0
*dh                  10.0
*[-son/+voice]       10.0
*aspiration          9.32

Predicted probabilities saved to results/TGT_pred.tsv


In [63]:
## M >> F DIF
cns = constraint_names
grs = [
    ["Ident (asp)", "Ident (voice)", "Ident (asp)/_V", "Ident (voice)/_V"], 
    ["*[+v][-v][+v]", "*dh", "*[-son/+voice]","*aspiration"]
]
mus = [5]
sms = [3]
msg = [[0, 1]]
diff_prior = DIFPrior(cns, grs, mus, sms, msg)

## Perform learning with regularization and returns the final weights
new_weights, learning_history = me.SGD_learn(violations, observed_probs, 1, 10000, 0.05, diff_prior)

print("Weights after learning:")
for cn in zip(cns, new_weights):
    print(f'{cn[0]:<20} {round(cn[1][0], 2)}')
print()

file_name = "results/DIF_pred_1.csv"
prob_prediction_to_csv(file_name, underlying_forms, candidates, violations, new_weights)
print(f"Predicted probabilities saved to {file_name}")

Weights after learning:
Ident (asp)          9.67
Ident (voice)        4.44
Ident (asp)/_V       6.14
Ident (voice)/_V     5.32
*[+v][-v][+v]        5.92
*dh                  6.73
*[-son/+voice]       2.81
*aspiration          5.11

Predicted probabilities saved to results/DIF_pred_1.csv


In [64]:
## M >> F DIF
cns = constraint_names
grs = [
    ["Ident (asp)"], 
    ["Ident (voice)"], 
    ["Ident (asp)/_V"], 
    ["Ident (voice)/_V"], 
    ["*[+v][-v][+v]", "*dh", "*[-son/+voice]", "*aspiration"]
]
mus = [3, 3, 3, 5]
sms = [1, 1, 1, 3]
msg = [[0, 1], [1, 2], [2, 3], [3, 4]]
diff_prior = DIFPrior(cns, grs, mus, sms, msg)

## Perform learning with regularization and returns the final weights
new_weights, learning_history = me.SGD_learn(violations, observed_probs, 1, 10000, 0.05, diff_prior)

print("Weights after learning:")
for cn in zip(cns, new_weights):
    print(f'{cn[0]:<20} {round(cn[1][0], 2)}')
print()

file_name = "results/DIF_pred_2.csv"
prob_prediction_to_csv(file_name, underlying_forms, candidates, violations, new_weights)
print(f"Predicted probabilities saved to {file_name}")

Weights after learning:
Ident (asp)          13.8
Ident (voice)        10.8
Ident (asp)/_V       7.81
Ident (voice)/_V     4.83
*[+v][-v][+v]        0.0
*dh                  0.0
*[-son/+voice]       0.0
*aspiration          0.0

Predicted probabilities saved to results/DIF_pred_2.csv
